In [1]:
import json # used for managing the JSON files from API
from urllib.request import urlopen # fetch URl of API
from pprint import pprint # just for printing values for human use
from copy import deepcopy

In [2]:
# Loads the papers to json from text file

with open("./stored_authors/papers.txt", encoding="utf8") as papers_file:
    papers = eval(papers_file.read())

In [3]:
authors_ids = {}
with open("./stored_authors/authors_ids.txt", encoding="utf8") as authors_file:
    authors_ids = eval(authors_file.read())
# pprint(authors_ids)

In [4]:
pprint(papers[0])

{'DOI': '10.1002/adem.200500223',
 'authors': ['75dc757852ded186f04bf13eae66c53803db5c82',
             '25e2798b6f1c8d5ca03eda52e347d33da845514f',
             '95f1bf7d79db23a55c6d8c155ecb3a6c2ec533ca',
             '7653e3bbcd863a9e7e7e142c07df15292a347874',
             '979826cedb64012e6ac7dd2787fcea904501fddc',
             '8e693fc4e5de76ff86bafd4577f7cd7a8556f74c'],
 'journal': 'Advanced Engineering Materials',
 'title': 'Improved Hydrogen Storage in the Metal-Organic Framework Cu3(BTC)2'}


In [5]:
# Just for visual clarity, not used for processing

papers_with_names = {}
for paper in papers:
    papers_with_names[ paper["DOI"] ] = []
    for person in paper["authors"]:
        papers_with_names[ paper["DOI"] ].append(authors_ids[person])
pprint(papers_with_names["10.1002/adem.200500223"])

[['P.', '', 'Krawiec'],
 ['Markus', '', 'Kramer'],
 ['M.', '', 'Sabo'],
 ['R.', '', 'Kunschke'],
 ['Heidrun', '', 'Fröde'],
 ['Stefan', '', 'Kaskel']]


In [6]:
duplicate_authors = {}
with open("./stored_authors/duplicate_authors.txt") as dup_file:
    duplicate_authors = eval(dup_file.read())

In [7]:
unknown_authors = {}
with open("./stored_authors/unknown_authors.txt") as unk_file:
    unknown_authors = eval(unk_file.read())

In [21]:
# not sure about this. Generate a coauthors dict? Only for unknown authors for right now

coauthors = {}


# Iterates through papers 
for paper in papers:
    
    # Iterates through authors
    for person in paper["authors"]:
        if person == "ORC_ID1234567890":
            print("WARNING")
        if person in unknown_authors or any(person in sublist for sublist in unknown_authors.values()): ## [DONE] need to change second part since values() is a list of lists
            
            # There is some problem here, I don't know what exactly though
            
            # Creates list if the person doesn't already exist
            if not person in coauthors:
                coauthors[person] = []
                
            coauthors[person].extend(paper["authors"])  # Adds all the coauthors to the list
            coauthors[person].remove(person)            # Removes the person itself from the coauthors list
            
# # Iterates through coauthors_list to make all of its lists sets (to remove duplicate coauthors) Should I be doing this?
# for person in coauthors:
# #     pprint(sorted(coauthor_list))
#     coauthors[person] = set(coauthors[person])
# #     pprint(coauthor_list)

# I probably shouldn't do the above. More duplicates -> higher count -> more probability of being matches?

# pprint(coauthors)

In [9]:
def find_key(dictionary, val):
    for key, value in dictionary.items():
        if value == val:
            return key
    return -1

In [10]:
# Testing whether every author is associated with a paper

testing = []

# for paper in papers:
#     for person in paper["authors"]:
#         if person in unknown_authors and person in unknown_authors.values():
#             testing.add(person)

pprint(unknown_authors.values())

# for person in testing:
#     pprint(coauthors[person])
#     print("meep")
# for person in testing:
#     pprint(coauthors[person])
#     print("boop")
    
# for person in unknown_authors:
#     if person in testing:
#         print("yes")
#     else: 
#         print("nooooooooooooooooooooooooooooooooooooooooooooooooooo")
        
# Lists all listed authors that are in both unknown and potential people i.e. coauthors the two share.
# Better method than what I used earlier?
# for person in testing:
#     if person in unknown_authors and person in unknown_authors.values():
#         print("yes")
#     else: 
#         print("no")
#         # It seems that no one is in both??

pprint(testing)


# Apparently some authors don't feature in a paper? I'm not sure if this is intended. If so, I'm not sure why

dict_values([['b7c2c5fbb917762d934fdaec1a6c4f7272ea3254'], ['bc2921622b43cd2d587db6573dc50e879deb52f5', '4ea9be7035978d170942261d6fa1eb3082fd8f88'], ['e6fdf689a7cf10e7ff672a0e9b433320587b8ca4', 'cd5f394013a324a9680394e6d268142b999b19d9', '269846570d6c133c19e2ae44b54bd24a012811fe'], ['c4aa2acfa847b763f573e4c50bc5c24418fd59d3'], ['966bbe37507c538696f134495ca6d67b360b64a2'], ['ccf2c7040ab38305c8f8b7f19398fb5852dfd250'], ['624566d921f2ced905a2803e3b30a08a990ddafc'], ['52b9ec426c110236605026615b0db869a094b255'], ['0000-0002-2980-7997'], ['318018bb75fc2ac1be4f09dc99a5aa80a73a9530'], ['717aa483598e57fa8d43b334631e02b73b4e14d9', '4cff47146053b704a1afc1fefc40d0c9a80dcfa5', 'ffaa1f073627cf457cf3093f74ea4160c66e6e9c', '357cdce51dc81cd578955fd5ae4aa77e53438e35', 'e27b01af94808ff484b8c2676c68ba63aeb2def8', '30c4174ad5406565d1464edd3d93d390b90aeec3', '128f7dc951fc34f5ae5f9cedf33e7ae194e335d3', '39f634d247f33ac9db147655b5f8a31cf2a10563', '8b52d7e96185f2fafd009aef0d1ad540fb944774', 'c7e96a9db67969a346

In [20]:
coauthor_counts = []

# Iterates through unknown authors
for unknown, matches in unknown_authors.items():
    
    # Iterates through possible matches for each unknown
    for person in matches:
        if person == "ORC_ID1234567890":
            print("WARNING - " + unknown)
        # Checks if the unknown author is in a paper ???
        if unknown in coauthors:
            
            # Iterates through coauthors of unknown author
            for unk_coauthor in coauthors[unknown]:
                
                # Checks to make sure that the potential match for the unknown is in the coauthors list 
                
                # person in coauthors and 
                if unk_coauthor in coauthors[person]:
                    pprint(coauthors[person])
                    print("yes")
                    
# Are there people in the database who are not associated with a paper? [SOLVED?] - doesn't seem like it?

# 2019-06-14 9:14AM - Seems fixed now, returns more than just a couple of people. Do I still need the if checks?

['82c527c6719ccad7d0ac7c6f4e6e311d387f68dc',
 '0000-0002-8485-5676',
 '049463c4d09c487d10765340558cb1727987ca4f',
 '5b738ab39ec1bc53eae5664f5ee21cbb703dca55',
 '0000-0002-8485-5676',
 'c527f49436fadf2137de52e650f9ffc9dc7b1cad',
 '7bd2af5ca5b3d133d652a66e9f45116787c06de8',
 '5b738ab39ec1bc53eae5664f5ee21cbb703dca55',
 '0000-0002-8485-5676',
 'c527f49436fadf2137de52e650f9ffc9dc7b1cad',
 '7bd2af5ca5b3d133d652a66e9f45116787c06de8',
 'facd36448a058852b76f2aea2ba59eddbd5526ca',
 'fc32b3e3e9dc298e703dd04579cb1c076102f212',
 '0949d922f7790bab76b314959401e3c436da043c',
 '5d024536b8f653b74b29c3b1adaf7265f26eaa5c',
 '5190a5a927aa4b043b10c1e8b836d1417edbf05f',
 '7bd2af5ca5b3d133d652a66e9f45116787c06de8',
 '1c38df94ad0e2c3ffcacbbd52aca2b0673826d56',
 '02fde538fd12d5c251e235b482e890d47f998339',
 '3dfcdfbe2e9f442a1f25c2ab7518e95c096f91e4',
 '7bd2af5ca5b3d133d652a66e9f45116787c06de8',
 '049463c4d09c487d10765340558cb1727987ca4f',
 '02fde538fd12d5c251e235b482e890d47f998339',
 'a5d8fffa669b64dc5d4ee5f1a1

KeyError: 'ORC_ID1234567890'

In [12]:
# if "75dc757852ded186f04bf13eae66c53803db5c82" in unknown_authors:
#     pprint(unknown_authors)

In [22]:
# Test cell

for author_id, name in authors_ids.items():
    if "Ribeiro" in name and "Ana" in name:
        print(author_id)
print()
for paper in papers:
    if "8b80ee24d983f356b3117484ac1a6f9a2e7ce441" in paper["authors"]:
        print(paper["DOI"])

831da78505f36a7f7f27627e3cf7c7ce70f3c814
8b80ee24d983f356b3117484ac1a6f9a2e7ce441

10.1002/ceat.201300046
10.1007/s10450-014-9639-3
10.1016/j.ces.2008.07.017
10.1016/j.micromeso.2011.06.024
10.1016/j.micromeso.2011.12.043
10.1016/j.seppur.2012.02.023
10.1016/j.seppur.2012.10.009
10.1016/j.seppur.2013.03.009
10.1016/j.seppur.2015.06.012
10.1021/acs.energyfuels.5b00975


In [14]:
# Test cell

for author_id, name in authors_ids.items():
    if "Krawiec" in name:
        print(author_id)
print()
for paper in papers:
    if "75dc757852ded186f04bf13eae66c53803db5c82" in paper["authors"]:
        print(paper["DOI"])

6517995a99763fe7f2bd80ac3a5672518323910f
75dc757852ded186f04bf13eae66c53803db5c82
ORC_ID1234567890

10.1002/adem.200500223
